# Word2vec

**Word2vec** is a probabilistic method created by [Mikolov et al., 2013]. It is a software package that actually includes:

$\quad$ - **2 algorithmms**: continuous bag-of-words (CBOW) and skip-gram. CBOW aims to predict a center word from the surrounding context in terms of word vectors. Skip-gram does the opposite, and predicts the *distribution*  (probability) of context words from a center word. In this Jupyter notebook, the second algorithm is implemented.

$\quad$ - **2 training methods**: negative sampling and hierarchical softmax. Negative sampling defines an objective by sampling *negative* examples, while hierarchical softmax defines an objective using an efficient tree structure to compute probabilities for all the vocabulary. In this Jupyter notebook, the negative sampling is implemented.

### Skip-Gram model

Notation for Skip-Gram Model:

*   $w_i$: Word $i$ from vocabulary $V$
*   $\mathcal{V} \in \mathbb{R^{n \times |V|}}$: Input word matrix
*   $v_i$: $i$-th column of $\mathcal{V}$, the input vector representation of word $w_i$
*   $\mathcal{U} \in \mathbb{R^{n \times |V|}}$: Output word matrix
*   $u_i$: $i$-th row of $\mathcal{U}$, the output vector representation of word $w_i$







Let's breakdown the way this model works in these 6 steps:
1.   We generate our one hot input vector $x \in \mathbb{R^{|V|}}$ of the center word
2.   We get our embedded word vector for the center word $v_c=\mathcal{V}x \in \mathbb{R^{|V|}}$
3.   Generate a score vector $z = \mathcal{U}v_c$.
4.   Turn the score vector into probabilities, $\hat{y} = softmax(z)$. Note that $\hat{y}_{c-m},...,\hat{y}_{c-1}, \hat{y}_{c+1},...,\hat{y}_{c+m}$ are the probabilities of observing each context word.
5.   We desire our probability vector generated to match the true probabiliies which is $y^{c-m},...,y^{c-1}, y^{c+1},...,y^{c+m}$, the one hot vectors of the actual output.



Objective function to minimize:

\begin{align}
   minimizeJ = - \sum_{j=0,j \ne m}^{2m}u_{c-m+j}^Tv_c + 2mlog\sum_{k=1}^{|V|}exp(u_k^Tv_c)
\end{align}

Loss functions in both algorithms are expensive to compute because of the softmax normalization, where we sum over all $|V|$ scores.

Negative Sampling:

For every training step, instead of looping over the entire vocabulary, we can just sample several negative examples! We 'sample' from a noise distribution ($P_n(w)$) whose probabilities match the ordering of the frequency of the vocabulary. To augment our formulation of the problem to incorporate Negative Sampling, all we need to do is update:

*   objective function
*   gradients
*   update rules

Mikolov et al. present Negative Sampling in Distributed Representations of Words and Phrases and their Compositionality. While negative sampling is based on the Skip-Gram model, it is in fact optimizing a different objective. Consider a pair
($w, c$) of word and context.

Did this pair come from the training data? Let’s denote by $P(D = 1|w, c)$ the probability that ($w, c$) came from the corpus data. Correspondingly, $P(D = 0|w, c)$ will be the probability that ($w, c$) did not come from the corpus data. First, let’s model $P(D = 1|w, c)$ with the sigmoid function:

\begin{align}
   P(D = 1|w, c, \theta) = \sigma(v_c^Tv_w)=\frac{1}{1+exp{(-v_c^Tv_w)}}
\end{align}

Now, we build a new objective function that tries to maximize the probability of a word and context being in the corpus data if it indeed is, and maximize the probability of a word and context not being in the corpus data if it indeed is not. We take a simple maximum likelihood approach of these two probabilities. The maximizing the likelihood is the same as minimizing the negative log likelihood, then we have:

\begin{align}
   J = - \sum_{(w,c) \in D}log\frac{1}{1+exp{(-u_w^Tv_c)}} - \sum_{(w,c) \in \tilde{D}}log\frac{1}{1+exp{(u_w^Tv_c)}}
\end{align}

Note that $\tilde{D}$ is a 'false' or 'nagative' corpus. Where we would have sentences like 'stock boil fish is toy'. Unnatural sentences that should
get a low probability of ever occurring. We can generate $\tilde{D}$ on the fly by randomly sampling this negative from the word bank.

So, for Skip-Gram Model we have a new objective function for observing the context word $c-m+j$ given the center word $c$ would be:

\begin{align}
   -log \ \sigma(u_{c-m+j}^Tv_c) - \sum_{k=1}^Klog \ \sigma(-\tilde{u}_k^Tv_c)
\end{align}

In the above formulation, {$\tilde{u}_k|k=1...K$} are sampled from $P_n(w)$. Let’s discuss what $P_n(w)$ should be. While there is much discussion of what makes the best approximation, what seems to work best is the Unigram Model raised to the power of $3/4$. Why $3/4$? Here’s an example that might help gain some intuition:

*   is: $0.9^{3/4} = 0.92$
*   Constitution: $0.09^{3/4} = 0.16$
*   bombastic: $0.01^{3/4} = 0.032$

"Bombastic" is now 3x more likely to be sampled while "is" only went up marginally. 